### Objetivo

Probar que el modelo permite identifiar sujetos cercanos a la semilla -- es decir, les asigna un score de lookalike alto

El procedimiento consiste en:
- Ocultar un proporción `PERC_TO_HIDE` de la semilla: estos van a ser parte de la población durante el entrenamiento, por lo cual tendrán 0 en el target
- Ejecutar el algoritmo de bagging que scorea a todos los sujetos de la población (incluyendo los ocultos)
- Evaluar la posición de los sujetos ocultos en la distribución del score. Esto se puede hacer visualmente o con estadísticos del estilo KS, AUROC, Average Precision, etc.

In [ ]:
ID_COLS = ['rut']
SEMILLA = 999
PERCT_TO_HIDE = 0.5 # proporcion de seed to hide
BAG_ITER = 100 # iteraciones de bagging

In [ ]:
import sys
# from ... import bagged_clf 

In [ ]:
import numpy as np
import pandas as pd

from sklearn.tree import train_test_split
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import seaborn as sns

# from ... import bagged_clf 

### Data

In [ ]:
# read     
X_seed = pd.read_pickle("data/X_seed.pkl")
X_poblacion = read_pickle("data/X_poblacion.pkl")

# drop IDS
X_seed = X_seed.drop(ID_COLS, axis=1)
X_poblacion = X_poblacion.drop(ID_COLS, axis=1)

### Simulación

Escondemos los sujetos de la semilla

In [ ]:
# split seed-hidden
np.random.seed(SEMILLA)
X_seed_sim, X_hidden = train_test_split(X_seed, test_size=PERC_TO_HIDE)
# poblacion
X_poblacion_sim = pd.concat([X_poblacion, X_hidden], ignore_index=True)
# indicador de hidden
flag_hidden = np.full(X_poblacion_sim.shape[0], 1)
flag_hidden[:X_poblacion.shape[0]] = 0

Definimos un clasificador base y corremos el algoritmo de baggin

In [ ]:
### Base Classifier
clf = DecisionTreeClassifier
clf_param = {'min_sample_leaf':1, 'random_state': SEMILLA}

In [ ]:
### Lookalike Scores
scores_sim, _ = bagged_clff(
    X_seed_sim, X_poblacion_sim, Classifier=clf,
    random_state=SEMILLA, T=BAG_ITER,
    **clf_param)

### Resultados

Visualizamos la distribucion del score en los *Hidden* y en el resto de la población

In [ ]:
sns.distplot(scores_sim[flag_hidden==0]), color='red', label='Resto') 
sns.distplot(scores_sim[flag_hidden==1]), color='skyblue', label='Hidden') 
plt.title('Distribucion del score')
plt.legend()

In [ ]:
sns.distplot(scores_sim[(flag_hidden==0) & (scores_sim>0)]), color='red', label='Resto') 
sns.distplot(scores_sim[(flag_hidden==0) & (scores_sim>0)]), color='skyblue', label='Hidden') 
plt.title('Distribucion del score')
plt.legend(